In [1]:
from __future__ import print_function 
import os, sys, h5py
import pandas as pd
import numpy as np
from six.moves import cPickle
from collections import OrderedDict
import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import utils, fit, init, visualize, saliency
import matplotlib.pyplot as plt
%matplotlib inline

### Trying to work on SRSF1 protein to build a best model that will be tested on other proteins

In [10]:
# load targets
#SRSF1 ids = RNCMPT00106,RNCMPT00107,RNCMPT00108,RNCMPT00109,RNCMPT00110,RNCMPT00163
filepath = '/Users/praveenanand/Documents/LabNoteBook/Peter_DeepNeural_RNABinding/Datasets/DeepBind/nbt3300-code/data/rnac'
df = pd.read_csv(os.path.join(filepath,'targets.tsv'), sep='\t')
names = list(df.columns.values)


175

In [11]:
names.index("RNCMPT00047")

188

##### Will be using index X that stands for RNCMPT00106 for exploring the model parameters. The Pearson correlation for the intensities in the deepbind model is around 0.376. 

In [2]:
def load_simulation(filepath):
    # setup paths for file handling

    trainmat = h5py.File(filepath, 'r')

    print("loading training data")
    X_train = np.array(trainmat['setA_seq']).astype(np.float32)
    y_train = np.array(trainmat['setA_targets']).astype(np.float32)

    print("loading test data")
    X_test = np.array(trainmat['setB_seq']).astype(np.float32)
    y_test = np.array(trainmat['setB_targets']).astype(np.float32)


    X_train = np.expand_dims(X_train, axis=3).transpose([0,2,3,1])
    X_test = np.expand_dims(X_test, axis=3).transpose([0,2,3,1])

    return (X_train, y_train), (X_test, y_test)


# load data
filename = 'rnac_SetAB_with_nan.hdf5'
data_path = '/Users/praveenanand/Documents/LabNoteBook/Peter_DeepNeural_RNABinding/Datasets/DeepBind/nbt3300-code/data/rnac'
filepath = os.path.join(data_path, filename)
raw_train, raw_test = load_simulation(filepath)


loading training data
loading test data


##### Will be loading in the test and the train dataset for RNCMPT00106 that has an index of 6. Common function to generate the log normalized test and train data for given protein